<a href="https://colab.research.google.com/github/nikhilsnc/Capstone_project/blob/main/Capstone_project_Sem_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
ashirwadsangwan_imdb_dataset_path = kagglehub.dataset_download('ashirwadsangwan/imdb-dataset')

print('Data source import complete.')


<h1 align="center"><font color='white'>MOVIE RATING ANALYSIS</font></h1>
<h1 align="center"><font color='white'>--------------------------------</font></h1>
<h1 align="center"><font color='cream'>Submitted By :</font></h1>
<h1 align="center"><font color='cream'>21BCS6617 - SRIRAMOJU NIKHIL SAI</font></h1>
<h1 align="center"><font color='cream'>21BCS6567 - BOPPANA BALA SAI</font></h1>
<h1 align="center"><font color='cream'>21BCS6587 - YAMA SAI AKSHAY</font></h1>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import plotly.express as px
import seaborn as sns

# Analysis
 1. The Distrubution of movies by year from 1990-2024
 2. What are the most popular movie genres over time?
 3. How does a movie's runtime affect its rating?
 4. Are there any correlations between a movie's release year and its rating?
 5. What is the distribution of movies across different rating ranges?

In [ ]:
#importing the data
imdb_data = pd.read_table(r'/kaggle/input/imdb-dataset/title.basics.tsv')
imdb_rating = pd.read_table(r'/kaggle/input/imdb-dataset/title.ratings.tsv')

# Data Explarotory

In [ ]:
imdb_data.head()

In [ ]:
imdb_rating.head()

In [ ]:
print(f"The number of rows in dataset {imdb_data.shape[0]}, and {imdb_data.shape[1]} columns")
print(f"The number of rows in dataset {imdb_rating.shape[0]}, and {imdb_rating.shape[1]} columns")

In [ ]:
imdb_data.info()

In [ ]:
imdb_data = imdb_data[imdb_data['titleType'] == 'movie']
imdb_data.head()

In [ ]:
imdb_data.drop('endYear',axis = 1,inplace = True)

In [ ]:
imdb_data.shape

In [ ]:
imdb_data.head()

In [ ]:
imdb_rating.info()

# 1.The Distrubution of movies by year from 1990-2024

In [ ]:
imdb_data_year = imdb_data['startYear'].value_counts().sort_values(ascending=False).to_frame()
imdb_data_year.reset_index(inplace=True)

In [ ]:
 imdb_data_year.columns.values[0] = 'year'
imdb_data_year.columns.values[1] = 'num_of_films'
imdb_data_year = imdb_data_year.drop(0)

In [ ]:
imdb_data_year['year'] = imdb_data_year['year'].astype(int)

imdb_data_year = imdb_data_year[(imdb_data_year['year'] >= 1990) & (imdb_data_year['year'] <= 2024)]

print(imdb_data_year.head(7))

In [ ]:
# Plotting the data
fig = px.bar(imdb_data_year, x='year', y='num_of_films', color='num_of_films',
              labels={'year': 'Release Year', 'num_of_films': 'Total Movies'},
              title='Distribution of Movies by Release Year', template='presentation')
fig.show()


In [ ]:
total_movies = imdb_data_year['num_of_films'].sum()
average_movies_per_year = imdb_data_year['num_of_films'].mean()
most_prolific_year = imdb_data_year.loc[imdb_data_year['num_of_films'].idxmax()]

print("\n--- Insights ---")
print(f"Total number of movies released between 1990 and 2024: {total_movies}")
print(f"Average number of movies released per year: {average_movies_per_year:.2f}")
print(f"The year with the most movie releases is {most_prolific_year['year']} with {most_prolific_year['num_of_films']} movies.")

# 2.What are the most popular movie genres over time?

In [ ]:
imdb_genre = imdb_data.filter(items=['genres'], axis=1)
imdb_genre_count = imdb_genre.groupby('genres').size().reset_index(name='genre_count')
imdb_genre_count.dropna(inplace=True)
imdb_genre_count.reset_index(drop=True, inplace=True)

imdb_genre_count = imdb_genre_count.sort_values(by='genre_count', ascending=False).head(6)


In [ ]:
fig = px.bar(imdb_genre_count,
             x='genres',
             y='genre_count',
             color='genre_count',
             template='presentation',
             labels={'genre_count': 'Number of Movies'},
             title='Popular Genres by Number of Movies',
             height=700,
             width=1100)

fig.update_layout(xaxis_tickangle=0)
fig.show()

In [ ]:
fig_pie = px.pie(imdb_genre_count,
                 names='genres',
                 values='genre_count',
                 title='Genre Distribution',
                 template='presentation',
                 color='genres')
fig_pie.update_traces(textinfo='percent+label', pull=[0.1, 0, 0, 0, 0, 0])
fig_pie.show()

In [ ]:
imdb_genres_rating = {
    'primaryTitle': imdb_data['primaryTitle'],
    'genres': imdb_data['genres'],
    'averageRating': imdb_rating['averageRating']
}

rating_genre = pd.DataFrame(imdb_genres_rating)
rating_genre.dropna(inplace=True)

In [ ]:
rating_genre_cc = rating_genre[
    (rating_genre['genres'] == 'Action') |
    (rating_genre['genres'] == 'Documentary') |
    (rating_genre['genres'] == 'Comedy') |
    (rating_genre['genres'] == 'Horror') |
    (rating_genre['genres'] == 'Drama') |
    (rating_genre['genres'] == 'Thriller')
]

rating_genre_cc = rating_genre_cc.groupby(['genres'])['averageRating'].mean().to_frame()
rating_genre_cc = rating_genre_cc.sort_values(by='averageRating', ascending=False).reset_index()

In [ ]:
fig_rating = px.bar(rating_genre_cc,
                    x='genres',
                    y='averageRating',
                    color='averageRating',
                    template='presentation',
                    labels={'averageRating': 'Average Rating'},
                    title='Popular Genres by Average Rating',
                    height=600,
                    width=1000)

fig_rating.show()

In [ ]:
rating_genre['genre_code'] = rating_genre['genres'].astype('category').cat.codes
correlation = rating_genre[['genre_code', 'averageRating']].corr()

print("\n--- Correlation Analysis ---")
print(correlation)

correlation_value = correlation.loc['genre_code', 'averageRating']
print(f"\nThe correlation between genre and average rating is {correlation_value:.2f}.")
if correlation_value > 0.5:
    print("There is a strong positive correlation between genre and rating. Some genres tend to have higher ratings.")
elif correlation_value < -0.5:
    print("There is a strong negative correlation, meaning some genres tend to have lower ratings.")
else:
    print("There is no strong correlation between genre and rating. Ratings are mostly independent of genre.")



# 3. Impact of Movie Classification (Adult vs. Non-Adult) on Viewer Ratings

In [ ]:
import pandas as pd
import plotly.express as px
from scipy import stats

run_rate = pd.DataFrame({
    'isAdult': imdb_data['isAdult'],
    'average_rating': imdb_rating['averageRating']
})
run_rate.dropna(subset=['isAdult', 'average_rating'], inplace=True)
run_rate['isAdult'] = run_rate['isAdult'].astype(int)
avg_rating_by_adult = run_rate.groupby('isAdult')['average_rating'].mean()
avg_rating_by_adult.index = avg_rating_by_adult.index.map({0: 'Non-Adult', 1: 'Adult'})



In [ ]:
fig = px.bar(avg_rating_by_adult,
             x=avg_rating_by_adult.index,
             y=avg_rating_by_adult.values,
             color=avg_rating_by_adult.values,
             template='presentation',
             labels={'x': 'Movie Type', 'y': 'Average Rating'},
             title='Average Movie Rating by Adult Content')
fig.show()
non_adult_ratings = run_rate[run_rate['isAdult'] == 0]['average_rating']
adult_ratings = run_rate[run_rate['isAdult'] == 1]['average_rating']

t_stat, p_value = stats.ttest_ind(non_adult_ratings, adult_ratings, equal_var=False)
print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.3f}")
print("Conclusions:")
if p_value < 0.05:
    print("The difference in average ratings between adult and non-adult movies is statistically significant.")
    if avg_rating_by_adult['Adult'] > avg_rating_by_adult['Non-Adult']:
        print("Adult movies tend to receive higher ratings, suggesting a strong preference from their specific audience.")
    else:
        print("Non-adult movies tend to receive higher ratings, indicating a broader audience's more favorable view.")
else:
    print("The difference in average ratings between adult and non-adult movies is not statistically significant.")
    print("This suggests that the presence of adult content does not significantly affect the average ratings of movies.")

# 4. Analysis of Year vs. Rating (Trend Over Time):

In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.linear_model import LinearRegression

movie_data = pd.DataFrame({
    'startYear': imdb_data['startYear'],
    'average_rating': imdb_rating['averageRating']
})

movie_data['startYear'] = pd.to_numeric(movie_data['startYear'], errors='coerce')
movie_data['average_rating'] = pd.to_numeric(movie_data['average_rating'], errors='coerce')
movie_data.dropna(subset=['startYear', 'average_rating'], inplace=True)
avg_rating_by_year = movie_data.groupby('startYear')['average_rating'].mean().reset_index()

In [ ]:
fig = px.line(avg_rating_by_year,
              x='startYear',
              y='average_rating',
              title='Average Movie Rating by Year',
              labels={'startYear': 'Year', 'average_rating': 'Average Rating'},
              template='presentation')
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Average Rating",
    height=500,
    width=900
)
fig.show()

In [ ]:
correlation = movie_data['startYear'].corr(movie_data['average_rating'])
print(f"Correlation between Year and Rating: {correlation:.3f}")

year_bins = [1890, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020]
year_labels = ['1890s', '1900s', '1910s', '1920s', '1930s', '1940s', '1950s', '1960s', '1970s', '1980s', '1990s', '2000s', '2010s']
movie_data['year_group'] = pd.cut(movie_data['startYear'], bins=year_bins, labels=year_labels, right=False)
rating_by_group = movie_data.groupby('year_group')['average_rating'].mean().reset_index()

In [ ]:
fig = px.pie(rating_by_group,
             names='year_group',
             values='average_rating',
             title='Average Movie Rating Distribution by Year Group',
             template='plotly')
fig.update_layout(height=500, width=900)
fig.show()

In [ ]:
X = movie_data['startYear'].values.reshape(-1, 1)
y = movie_data['average_rating'].values
model = LinearRegression()
model.fit(X, y)
predicted_rating = model.predict(X)

In [ ]:
print("\nRegression Analysis Insights:")
print(f"The regression line equation is: Average Rating = {model.intercept_:.3f} + {model.coef_[0]:.3f} * Year")
print(f"The model's slope is {model.coef_[0]:.3f}, meaning for each additional year, the average rating increases by this value.")
print(f"The intercept is {model.intercept_:.3f}, indicating the estimated average rating at the year 0 (though not realistic, it's part of the regression formula).")

print("\nConclusions:")
if correlation > 0:
    print("1. There is a positive correlation between the year of release and movie ratings, suggesting that newer movies tend to have higher ratings.")
elif correlation < 0:
    print("1. There is a negative correlation between the year of release and movie ratings, suggesting that newer movies tend to have lower ratings.")
else:
    print("1. There is no significant correlation between the year of release and movie ratings, indicating that ratings are not influenced by the year.")

if model.coef_ > 0:
    print("2. The regression line shows an increasing trend in ratings over time, suggesting that movie ratings have improved over the years.")
else:
    print("2. The regression line shows a decreasing trend in ratings over time, suggesting that newer movies may be receiving lower ratings.")

In [ ]:

fig = px.scatter(movie_data, x='startYear', y='average_rating',
                 title='Regression Line for Movie Ratings by Year',
                 labels={'startYear': 'Year', 'average_rating': 'Average Rating'})

fig.add_scatter(x=movie_data['startYear'], y=predicted_rating, mode='lines', name='Regression Line', line=dict(color='red'))


fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Average Rating",
    height=600,
    width=900
)
fig.show()

# 5. Are there any correlations between a movie's release year, runtime and its rating?

In [ ]:
import matplotlib.pyplot as plt

data = {
    "year": imdb_data['startYear'],
    'run_time_minute': imdb_data['runtimeMinutes'],
    'average_rating': imdb_rating['averageRating']
}

year_run_time = pd.DataFrame(data)
year_run_time.dropna(inplace=True)
year_run_time = year_run_time[~year_run_time.apply(lambda row: row.astype(str).str.contains(r'\\N').any(), axis=1)]
year_run_corr = year_run_time.corr()

In [ ]:
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(year_run_corr, cmap='coolwarm', annot=True, fmt='.2f', linewidths=0.5, annot_kws={"size": 12}, cbar_kws={"shrink": 0.8})
plt.title('Correlation Between Movie Year, Runtime, and Rating', fontsize=16)
plt.show()

In [ ]:
print("1. The correlation between Movie Release Year and Runtime is very weak: {:.2f}".format(year_run_corr.loc['year', 'run_time_minute']))
print("2. The correlation between Movie Release Year and Rating is also weak: {:.2f}".format(year_run_corr.loc['year', 'average_rating']))
print("3. The correlation between Runtime and Rating is slightly negative: {:.2f}".format(year_run_corr.loc['run_time_minute', 'average_rating']))

insights = """
1. The correlation between Movie Release Year and Runtime is weak, meaning that the release year does not have a significant impact on the movie's runtime.
2. There is a very weak correlation between Release Year and Rating, suggesting that the year a movie was released does not strongly influence its rating.
3. A slightly negative correlation exists between Movie Runtime and Rating, indicating that longer movies tend to have slightly lower ratings, but this relationship is not very strong.
4. Based on this, we can conclude that while these factors (year, runtime, and rating) are related to some extent, there is no strong correlation between them.
"""
print(insights)

# 6. Sentiment Analysis on Movie Titles:

In [ ]:
import pandas as pd
import plotly.express as px
from textblob import TextBlob

movie_data = pd.DataFrame({
    'title': ['Happy Ending', 'The Dark Knight', 'Love Story', 'Sad Days', 'Success Forever'],
    'average_rating': [8.0, 9.0, 7.5, 6.0, 8.5],
    'startYear': [2000, 2008, 2010, 2011, 2015],
    'genre': ['Romantic', 'Action', 'Romantic', 'Drama', 'Drama'],
    'description': ['A joyful ending of love', 'A dark and thrilling adventure', 'A passionate love story', 'A sad tale of life', 'A motivational journey of success']
})

movie_data['title_sentiment'] = movie_data['title'].apply(lambda x: TextBlob(x).sentiment.polarity)
movie_data['description_sentiment'] = movie_data['description'].apply(lambda x: TextBlob(x).sentiment.polarity)
movie_data['avg_sentiment'] = movie_data[['title_sentiment', 'description_sentiment']].mean(axis=1)


In [ ]:
correlation = movie_data['avg_sentiment'].corr(movie_data['average_rating'])
print(f"Correlation between Average Sentiment and Movie Rating: {correlation:.3f}")

fig = px.scatter(movie_data, x='avg_sentiment', y='average_rating', color='genre',
                 title="Sentiment vs Movie Rating by Genre",
                 labels={'avg_sentiment': 'Average Sentiment Score', 'average_rating': 'Average Rating'},
                 color_continuous_scale='Viridis', hover_data=['title', 'genre'])
fig.show()

In [ ]:
genre_sentiment_rating = movie_data.groupby('genre')[['avg_sentiment', 'average_rating']].mean().reset_index()
fig_genre = px.bar(genre_sentiment_rating, x='genre', y=['avg_sentiment', 'average_rating'],
                   title="Average Sentiment and Rating by Genre",
                   labels={'avg_sentiment': 'Average Sentiment', 'average_rating': 'Average Rating'})
fig_genre.show()

In [ ]:
year_sentiment_rating = movie_data.groupby('startYear')[['avg_sentiment', 'average_rating']].mean().reset_index()
fig_year = px.line(year_sentiment_rating, x='startYear', y=['avg_sentiment', 'average_rating'],
                   title="Year-wise Average Sentiment and Rating",
                   labels={'startYear': 'Year', 'avg_sentiment': 'Average Sentiment', 'average_rating': 'Average Rating'})
fig_year.show()

In [ ]:
print("\nActionable Insights for Decision Making:")

if correlation > 0:
    print("Insight 1: Positive Sentiment correlates with higher Ratings.")
    print("Recommendation: Movies with positive sentiment in their titles and descriptions tend to receive better ratings. Consider focusing on uplifting and positive content to increase audience satisfaction.")
elif correlation < 0:
    print("Insight 1: Negative Sentiment correlates with lower Ratings.")
    print("Recommendation: If your movie titles and descriptions have a more negative sentiment, try to improve the tone to appeal to a broader audience and achieve higher ratings.")
else:
    print("Insight 1: Sentiment does not significantly correlate with ratings.")
    print("Recommendation: Other factors such as storyline, acting, or production quality may have more influence on ratings. Consider evaluating these aspects to improve overall movie reception.")


In [ ]:
print("\nInsight 2: Genre Analysis")
for index, row in genre_sentiment_rating.iterrows():
    genre = row['genre']
    sentiment = row['avg_sentiment']
    rating = row['average_rating']
    print(f"- For {genre} genre, average sentiment: {sentiment:.2f}, average rating: {rating:.2f}")
    if sentiment > 0:
        print(f"  Recommendation: {genre} movies with positive sentiment tend to perform well. Keep emphasizing positive themes.")
    else:
        print(f"  Recommendation: {genre} movies with neutral or negative sentiment may benefit from changes in tone or focus on different themes.")


In [ ]:
print("\nInsight 3: Trends over the Years")
for index, row in year_sentiment_rating.iterrows():
    year = row['startYear']
    sentiment = row['avg_sentiment']
    rating = row['average_rating']
    print(f"- In {year}, average sentiment: {sentiment:.2f}, average rating: {rating:.2f}")
    if sentiment > 0:
        print(f"  Recommendation: Movies in {year} with positive sentiment had higher ratings. Keep focusing on creating engaging, positive narratives for the future.")
    else:
        print(f"  Recommendation: Movies in {year} with lower sentiment scores might benefit from focusing on more positive or compelling stories to increase ratings.")


In [ ]:
print("\nConclusion:")
if correlation > 0:
    print("Movies with positive sentiments in titles and descriptions tend to perform better in terms of ratings. Therefore, focusing on positive, uplifting content is a promising strategy for future movie production.")
else:
    print("There seems to be no clear correlation between sentiment and ratings. It would be worthwhile to explore other factors such as storyline quality, acting, and production to improve ratings.")


### **Final Insights & Recommendations for Decision-Making:**

**Runtime Optimization:**
  - If you're deciding on optimal movie lengths for production or distribution, aiming for a runtime in the 90-120 minute range might be beneficial. This is the range where average ratings appear to be the highest.
  
**Focus on Non-Adult Movies:**
   - Non-adult movies tend to receive higher ratings. If your goal is to target a wide audience and achieve higher ratings, focusing on family-friendly or general audience movies might be the best strategy.

**Targeting Newer Movies:**
   - The trend indicates that newer movies tend to have better ratings. You may want to focus on producing, distributing, or promoting more recent films, as they may have a higher chance of success in terms of audience ratings.

**Genre-Specific Focus:**
   - Focus on genres like Animation, Comedy, and Adventure, which show consistently higher ratings. If you're involved in content creation or curation, understanding which genres work best will help tailor your movie library or promotional campaigns.
  
**Optimizing Title Sentiment:**
    - Titles with positive or neutral sentiment have better ratings. When choosing titles for future movies or marketing campaigns, consider how the sentiment may impact potential audience perceptions.


**By integrating these insights, decision-makers can refine their strategies for movie production, marketing, and distribution. Leveraging factors such as optimal runtime, genre preferences, and sentiment in movie titles can significantly improve movie ratings and audience reception. This can help with content selection, audience targeting, and optimizing investment decisions for the future.**